# MODEL-PROJECT: The Black-Scholes(-Merton)-Model

**Before we start please install these packages in your VScode terminal (if not already installed)**

To do that just remove the '#' in the codecell below and run the code once. Be sure to put the '#' back in before you continue as to not run into problems when rerunning the code.

In [14]:
# !pip install yfinance
# !pip install ipywidgets
# !pip install dash
# !pip install dash-core-components

Imports and set magics:

In [15]:
import numpy as np
from scipy import optimize
import sympy as sp
import matplotlib.pyplot as plt
from scipy.stats import norm
import yfinance as yf
from dash import dcc
import ipywidgets as widgets
from IPython.display import display

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

The Black-Scholes model is a mathematical model for pricing European-style options. It's widely used in the financial industry, and it has significantly impacted the field of financial economics. The model assumes that the price of the underlying asset follows a geometric Brownian motion, and it derives a closed-form solution for the price of said asset. It provides a theoretical estimate of the value of a call or put option based on the following parameters:

-The current price of the underlying asset (S)  
-The strike price of the option (K)  
-The time to maturity (T)  
-The risk-free interest rate (r)  
-The volatility of the underlying asset (σ)  

The Black-Scholes equation for the price of a call option (C) and put option (P) is given by:

$ C(S, K, T, r, σ) = S N(d1) - K e^{-rT} N(d2) $

$ P(S, K, T, r, σ) = K e^{-rT} N(-d2) - S N(-d1) $

where:

$ d1 = \frac{log(\frac{S}{K}) + (r + \frac{σ^2}{2})T}{σ\sqrt{T}} $

$ d2 = d1 - σ\sqrt{T} $
 
and $N(x)$ is the cumulative distribution function of the standard normal distribution.

In this project, we'll implement the Black-Scholes model using Python and perform various analyses, including numerical solutions for different real world assets and visualizations.

In [16]:
# Define the Black-Scholes Model equation
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type == 'call':
        price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    
    return price

## Analytical solution

The Black-Scholes model has a closed-form solution, which we can obtain using Sympy. We'll first derive the steady-state equation and then use Sympy to solve and lambdify it.

In [17]:
# Define the Black-Scholes Model symbols
S, K, T, r, sigma = sp.symbols('S K T r sigma')

# Define the Black-Scholes Model equation
d1 = (sp.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*sp.sqrt(T))
d2 = d1 - sigma*sp.sqrt(T)

# Define lambdified functions for d1 and d2
d1_lambdified = sp.lambdify((S, K, T, r, sigma), d1, 'numpy')
d2_lambdified = sp.lambdify((S, K, T, r, sigma), d2, 'numpy')

# Define Black-Scholes call and put pricing functions
def black_scholes_call(S, K, T, r, sigma):
    d1_val = d1_lambdified(S, K, T, r, sigma)
    d2_val = d2_lambdified(S, K, T, r, sigma)
    return S*norm.cdf(d1_val) - K*np.exp(-r*T)*norm.cdf(d2_val)

def black_scholes_put(S, K, T, r, sigma):
    d1_val = d1_lambdified(S, K, T, r, sigma)
    d2_val = d2_lambdified(S, K, T, r, sigma)
    return K*np.exp(-r*T)*norm.cdf(-d2_val) - S*norm.cdf(-d1_val) 




## Numerical solution

We can also solve the Black-Scholes model numerically using optimization algorithms. In this case, we'll use the Newton-Raphson method.

In [18]:
# Define the Black-Scholes Model equation
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type == 'call':
        price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    
    return price

# Define the initial values for S, K, T, r, and sigma
S = 100
K = 100
T = 1
r = 0.05
sigma = 0.2

# Define a range of strike prices
strike_prices = np.linspace(S * 0.5, S * 1.5, 100)

# Create a function to update the plot with the new value of sigma
def update_plot(sigma):
    call_prices = [black_scholes(S, K, T, r, sigma, option_type='call') for K in strike_prices]
    put_prices = [black_scholes(S, K, T, r, sigma, option_type='put') for K in strike_prices]
    
    plt.figure(figsize=(10, 6))
    plt.plot(strike_prices, call_prices, label="Call prices")
    plt.plot(strike_prices, put_prices, label="Put prices")
    plt.title(f"Black-Scholes Model for S={S}, K={K}, T={T}, r={r}, sigma={sigma}")
    plt.xlabel("Strike Price")
    plt.ylabel("Option Price")
    plt.legend()
    plt.show()

# Create a slider widget for sigma
sigma_slider = widgets.FloatSlider(
    value=sigma,
    min=0.01,
    max=1,
    step=0.01,
    description='Sigma:',
    continuous_update=False
)

# Create an output widget to display the plot
out = widgets.Output()

# Display the interactive plot
widgets.interact(update_plot, sigma=sigma_slider)
display(out)


interactive(children=(FloatSlider(value=0.2, continuous_update=False, description='Sigma:', max=1.0, min=0.01,…

Output()

# Further analysis

Now, let's analyze how the model changes with different parameter values and visualize the results.

In [20]:
# List of 15 stock tickers 
stock_list = ["AAPL", "MSFT", "GOOG", "AMZN", "TSLA", "NFLX", "BRK-B", "JPM", "JNJ", "V", "WMT", "UNH", "PG", "NVDA", "HD"]
company_list = ["Apple", "Microsoft", "Google", "Amazon", "Tesla", "Netflix", "Berkshire Hathaway", "JPMorgan Chase", "Johnson & Johnson", "Visa", "Walmart", "UnitedHealth Group", "Procter & Gamble", "NVIDIA", "Home Depot"]

# Define dictionary to map companies to tickers
company_ticker_dict = {company: ticker for company, ticker in zip(company_list, stock_list)}

# Function to plot Black-Scholes model for a given stock ticker
def plot_black_scholes(company):
    with out:
        try:
            # Get stock data
            stock_ticker = company_ticker_dict[company]
            stock_data = yf.download(stock_ticker, progress=False)
            stock_price = stock_data.iloc[-1]["Close"]

            # Calculate call and put prices for a range of strike prices
            strike_prices = np.linspace(stock_price * 0.5, stock_price * 1.5, 100)
            call_prices = [black_scholes_call(stock_price, K, T, r, sigma) for K in strike_prices]
            put_prices = [black_scholes_put(stock_price, K, T, r, sigma) for K in strike_prices]

            # Clear the output and create the plot
            out.clear_output(wait=True)
            plt.figure(figsize=(10, 6))
            plt.plot(strike_prices, call_prices, label="Call prices")
            plt.plot(strike_prices, put_prices, label="Put prices")

            plt.title(f"Black-Scholes Model for {company}")
            plt.xlabel("Strike Price")
            plt.ylabel("Option Price")
            plt.legend()
            plt.show()
        except IndexError as e:
            print(f"Error: {e}. Data for {company} might not be available or the stock might be delisted.")

# Create a dropdown menu for company selection
company_dropdown = widgets.Dropdown(
    options=company_list,
    value=company_list[0],
    description="Company:",
)

# Create an Output widget to display the plot
out = widgets.Output()

# Display the interactive plot
widgets.interact(plot_black_scholes, company=company_dropdown)
display(out)


interactive(children=(Dropdown(description='Company:', options=('Apple', 'Microsoft', 'Google', 'Amazon', 'Tes…

Output()

In [ ]:
# Trying to get an actual sigma for the real world stock data. ChatGPT stuff that still needs to be refined:











# Define a list of ticker symbols for the stocks you want to analyze
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "FB"]

# Download data for all the tickers in the list
data = yf.download(tickers, start="2020-01-01", end="2020-12-31", group_by="ticker")

# Calculate the standard deviation of the Close prices for each stock
std_devs = data["Close"].std()

# Create a dictionary to store the results
results_dict = {}

# Loop through the list of tickers and store the standard deviation for each one in the dictionary
for ticker in tickers:
    results_dict[ticker] = std_devs[ticker]

# Print the dictionary to display the results
print(results_dict)

# Conclusion

Additionally, in this version of the project, we have created an interactive plot to visualize the Black-Scholes option prices for 15 different European stocks. The stocks were chosen to represent a diverse range of industries and countries. The interactive plot allows users to select a stock from a dropdown menu and view the corresponding option prices for the call and put options.